In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import datetime
import calendar
import zipfile
from multiprocessing.pool import ThreadPool

In [ ]:
deliverables_url = "https://www.bseindia.com/markets/equity/EQReports/GrossShortPos.aspx?flag=0"

In [ ]:
path = os.path.join(os.getcwd(),"Data\Deliverables")

def download_deliverables():
    """
    Downloads the deliverables of the current day.

    deliverables_url = "https://www.bseindia.com/markets/equity/EQReports/GrossShortPos.aspx?flag=0"

    creates the driver.

    opens the deliverables_url.
    
    downloads the file.

    Methods:
    --------

    create_driver : creates the chrome driver.

    download : downloads the file. invokes extract_save method.

    extract_save : extracts the file and renames it to the specified name.

    """

    def extract_save(name):
        """
        extracts the file and renames it to the specified name.

        Parameters
        ----------
        name : string
            name of the file i.e, security id

        """

        res = zipfile.ZipFile(os.path.join(path,name))
        res.extractall(path)
        deli = pd.read_csv(os.path.join(path,name),sep="|")
        today = datetime.date.today()-datetime.timedelta(days=1)
        output_name = str(today.day).zfill(2) + str(today.month).zfill(2)+str(today.year).zfill(4)
        deli.to_csv(os.path.join(path,output_name+".csv"),index=None)

    def download():
        """
        downloads the file.
        invokes the extract_save method.

        """
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_hylinksearch"]')
        element.click()
        url = element.get_attribute("href")
        name = url.split("/")[-1]
        time.sleep(3)
        driver.quit()
        extract_save(url.split("/")[-1])
        os.remove(os.path.join(path,name))
        os.remove(os.path.join(path,name.replace(".zip",".txt")))

    def create_driver():
        """
        Creates a Chrome Driver.

        Returns
        --------
        driver : driver
            chrome web driver.
        """
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    driver = create_driver()
    driver.get(deliverables_url)
    download()


In [ ]:
# download_deliverables()